In [42]:
# wtf happened to this file and why doesn't the torch dataloader find it?
# audiopath = "/usr/src/app/radtts/data/lj_data/LJSpeech-1.1/wavs/LJ027-0028_22k_normalized.wav"
# pitchpath = "/usr/src/app/radtts/data_cache/lj_data_LJSpeech-1.1_wavs_LJ027-0028_22k_normalized.wav_f0_sr22050_fl1024_hl256_f0min80.0_f0max640.0_log0.pt"


# audiopath = "/usr/src/app/radtts/data/lj_data/LJSpeech-1.1/wavs/LJ044-0174_22k_normalized.wav"
# pitchpath = "/usr/src/app/radtts/data_cache/lj_data_LJSpeech-1.1_wavs_LJ044-0174_22k_normalized.wav_f0_sr22050_fl1024_hl256_f0min80.0_f0max640.0_log0.pt"
import os
use_log_f0 = 0
audiopath = "/usr/src/app/radtts/data/lj_data/LJSpeech-1.1/wavs/LJ003-0239_22k_normalized.wav"
# pitchpath = "/usr/src/app/radtts/data_cache/lj_data_LJSpeech-1.1_wavs_LJ003-0239_22k_normalized.wav_f0_sr22050_fl1024_hl256_f0min80.0_f0max640.0_log0.pt"
filename = '_'.join(audiopath.split('/')[-4:])
betabinom_cache_path = "/usr/src/app/radtts/data_cache"
f0_path = os.path.join(betabinom_cache_path, filename)
f0_path += "_f0_sr{}_fl{}_hl{}_f0min{}_f0max{}_log{}.pt".format(
    sampling_rate, filter_length, hop_length,
    data_config['f0_min'], data_config['f0_max'], use_log_f0)

In [45]:
import pandas as pd
data = pd.read_csv('/usr/src/app/radtts/data/lj_data/LJSpeech-1.1/metadata_formatted_full_pitch.txt', sep = '|', index_col = None,header = None)

In [49]:
audiopaths = data[0]
for audiopath in audiopaths:
    
    filename = '_'.join(audiopath.split('/')[-4:])
    betabinom_cache_path = "/usr/src/app/radtts/data_cache"
    f0_path = os.path.join(betabinom_cache_path, filename)
    f0_path += "_f0_sr{}_fl{}_hl{}_f0min{}_f0max{}_log{}.pt".format(
        sampling_rate, filter_length, hop_length,
        data_config['f0_min'], data_config['f0_max'], use_log_f0)    
    
    if not os.path.exists(f0_path):
        print(f0_path)
        audio, sampling_rate = load_wav_to_torch(audiopath)
        f0, voiced_mask, p_voiced = get_f0_pvoiced(audio.cpu().numpy(), sampling_rate=sampling_rate, frame_length=filter_length,
                           hop_length=hop_length, f0_min=mel_fmin, f0_max=mel_fmax)
        torch.save({'f0': f0,
                    'voiced_mask': voiced_mask,
                    'p_voiced': p_voiced}, f0_path)

/usr/src/app/radtts/data_cache/lj_data_LJSpeech-1.1_wavs_LJ005-0094_22k_normalized.wav_f0_sr22050_fl1024_hl256_f0min80.0_f0max640.0_log0.pt
/usr/src/app/radtts/data_cache/lj_data_LJSpeech-1.1_wavs_LJ040-0007_22k_normalized.wav_f0_sr22050_fl1024_hl256_f0min80.0_f0max640.0_log0.pt


In [43]:
f0_path

'/usr/src/app/radtts/data_cache/lj_data_LJSpeech-1.1_wavs_LJ003-0239_22k_normalized.wav_f0_sr22050_fl1024_hl256_f0min80.0_f0max640.0_log0.pt'

In [33]:
from librosa import pyin
from uberduck_ml_dev.exec.train_radtts_with_ray import data_config, load_wav_to_torch

In [47]:
filter_length=data_config['filter_length']
hop_length=data_config['hop_length']
win_length=data_config['win_length']
sampling_rate=22050
n_mel_channels=data_config['n_mel_channels']
f0_min=data_config['f0_min']
f0_max=data_config['f0_max']

In [48]:
max_wav_value = 32768.

def get_f0_pvoiced(audio, sampling_rate=22050, frame_length=1024,
                   hop_length=256, f0_min=100, f0_max=300):

    audio_norm = audio / max_wav_value
    f0, voiced_mask, p_voiced = pyin(
        audio_norm, f0_min, f0_max, sampling_rate,
        frame_length=frame_length, win_length=frame_length // 2,
        hop_length=hop_length)
    f0[~voiced_mask] = 0.0
    f0 = torch.FloatTensor(f0)
    p_voiced = torch.FloatTensor(p_voiced)
    voiced_mask = torch.FloatTensor(voiced_mask)
    return f0, voiced_mask, p_voiced

In [36]:
import torch
audio, sampling_rate = load_wav_to_torch(audiopath)
f0, voiced_mask, p_voiced = get_f0_pvoiced(audio.cpu().numpy(), sampling_rate=sampling_rate, frame_length=filter_length,
                   hop_length=hop_length, f0_min=f0_min, f0_max=f0_max)

In [37]:
torch.save({'f0': f0,
            'voiced_mask': voiced_mask,
            'p_voiced': p_voiced}, pitchpath)

In [13]:
sampling_rate

22050

In [15]:
help(pyin)

Help on function pyin in module librosa.core.pitch:

pyin(y, fmin, fmax, sr=22050, frame_length=2048, win_length=None, hop_length=None, n_thresholds=100, beta_parameters=(2, 18), boltzmann_parameter=2, resolution=0.1, max_transition_rate=35.92, switch_prob=0.01, no_trough_prob=0.01, fill_na=nan, center=True, pad_mode='reflect')
    Fundamental frequency (F0) estimation using probabilistic YIN (pYIN).
    
    pYIN [#]_ is a modificatin of the YIN algorithm [#]_ for fundamental frequency (F0) estimation.
    In the first step of pYIN, F0 candidates and their probabilities are computed using the YIN algorithm.
    In the second step, Viterbi decoding is used to estimate the most likely F0 sequence and voicing flags.
    
    .. [#] Mauch, Matthias, and Simon Dixon.
        "pYIN: A fundamental frequency estimator using probabilistic threshold distributions."
        2014 IEEE International Conference on Acoustics, Speech and Signal Processing (ICASSP). IEEE, 2014.
    
    .. [#] De Chev